## Read earnings reports (OO version)

E. Quinn 12/22/2019

This notebook uses pdfminer to extract the information from the individual earnings report

The documentation for pdfminer is at:

https://buildmedia.readthedocs.org/media/pdf/pdfminer-docs/latest/pdfminer-docs.pdf

Maintenance:

* 3/6/2020  
  * Add check date and number
* 3/7/2020  
  * Align personnel classes with support professionals structure
  * Implement salary step capture for support professionals

## Import standard python datascience packages

In [1]:
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid

## Import pdfminer packages

In [3]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

### Show the directory we are running in

In [4]:
!pwd

/home/gquinn/EG/school_committee/finance_subcommittee/notebooks


## Read the pdf and create a dictionary with the contents of each text box

### Function read_pdf() reads a PDF and returns a dictionary containing the contents

Strategy for this document:  

Save information from each element in the LTTextBox objects in a dictionary including:

- x0 horizontal coordinate of the upper left corner of the text box
- x1 horizontal coordinate of the lower right corner of the text box
- y0 vertical coordinate of the upper left corner of the text box
- y1 vertical coordinate of the lower right corner of the text box
- page number 
- sequence number of text box within this page
- text contained in the text box, converted to ascii

Parsing the text is complicated by the fact that that a text box may span multiple columns and/or rows, and the text box groupings vary quite a bit depending on the page contents and layout.

However, with a bit of luck the structure of the document will allow the contents to be deciphered with the following heuristics:

- Text boxes containing left justified columns will tend to have nearly the same x0 coordinates
- Text boxes containing right justified columns will tend to have nearly the same x1 coordinates
- The codes for fund, account code, and object code are numeric and have fixed lengths
- Extraneous information is often preceded or followed by a series of underscore and newline characters
- Last name can be distinguished because is the only field that is all characters followed by a comma
- Last name may be preceded by between one and three numerical fields:  fund, account, object.  If it is, the x0 value is shifted to the left.
    - Three numerical fields precede the name:  assume they are fund, account, object
    - Two numerical fields precede the name: assume they are account, object
    - One numerical field precedes the name: assume it is object
    

In [5]:
def read_pdf(path):
    document = open(path, 'rb')                                     #read a pdf and create a document object
    rsrcmgr = PDFResourceManager()                                  #create a resource manager
    laparams = LAParams()                                           #set the parameters for analysis
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)          #create a PDF page aggregator object
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    pdf={}                                                          #dictionary to hold the results

    pageno = -1                                                     #initialize page coounter to zero

    for page in PDFPage.get_pages(document):                        #loop through the pdf page by page
        pageno = pageno + 1                                         #increment the page number
        pdf[pageno] = {}                                            #dictionary for this page
        interpreter.process_page(page)                              # receive the LTPage object for the page.
        layout = device.get_result()                                # create layout object
        tbox_no=0                                                   # index for element number
        for element in layout:
            if (type(element).__name__=='LTTextBoxHorizontal'):     #loop through text boxes
                tbox_no += 1                                        #increment text box number
                pdf[pageno][tbox_no] = {}                           #dictionary for text boxes within page
                x0 = round(element.x0,2)                            #x0 coordinate of textbox corner
                x1 = round(element.x1,2)                            #x1 coordinate of textbox corner
                y0 = round(element.y0,2)                            #y0 coordinate of textbox corner
                y1 = round(element.y1,2)                            #y1 coordinate of textbox corner
                txt = element.get_text().encode('ascii', 'ignore')  #text converted to ascii
                pdf[pageno][tbox_no]['x0'] = x0                     #create x0 coordinate entry
                pdf[pageno][tbox_no]['x1'] = x1                     #create x1 coordinate entry
                pdf[pageno][tbox_no]['y0'] = y0                     #create y0 coordinate entry
                pdf[pageno][tbox_no]['y1'] = y1                     #create y1 coordinate entry

                pdf[pageno][tbox_no]['text'] = ''.join(chr(c) for c in txt) #convert bytes to string
    return(pdf)

### Utility functions

In [6]:
#remove the commas from earnings and rate values

def remove_commas(st):
    newstr = st.replace(',','')                     #remove commas from string
    return(newstr)

In [7]:
#remove the headings fields 

def remove_headings(st):
    lines = st.split('\n')                         #split the string at newline characters '\n'
    for line in lines:                             #loop through the resulting lines
        if (line.startswith('FUND ') |\
           (line.startswith('POSITION')) |\
           (line.startswith('RATE')) |\
           (line.startswith('ACCT-')) |\
           (line.startswith('CHECK')) |\
           (line.startswith('_'))):                #check for strings that appear only in headings
            try:
                newline_index = st.index('\n')     #if present, remove this line from the text string
                st = st[newline_index+1:]
            except ValueError:
                print('Value Error',st)            #recover from Value Error and print string
        else:
            return(st)                             #if no headings, just return
    return('')

### Read the FY2017 earnings report

In [8]:
#p17 = read_pdf('../FY17 Gene_Redacted.pdf')
p17 = read_pdf('../FY17 Gene.pdf')

### Read the FY2018 earnings report

In [9]:
#p18 = read_pdf('../FY18 Gene_Redacted.pdf')
p18 = read_pdf('../FY18 Gene.pdf')

### Build a dictionary with only those text boxes containing names

Use the following algorithm to identify text boxes that contain names:

- x0, horizontal coordinate of the upper left corner of the text box, is less than 162
- the text string contains at least one comma

In [10]:
def get_names(dct):

    dnames = {}

    fund = ''
    acct = ''
    obj  = ''
    
    for page in sorted(dct.keys()):                                #loop through text box dictionary by page # 
        if (page not in dnames.keys()):                            #page number is highest level key
            dnames[page] = {}                                      #initialize entry for this page
        for tb in sorted(dct[page].keys()):                        #loop through all text boxes on this page
            if (dct[page][tb]['x0'] < 162.0):                      #those with names start to the left of x0=162
                txt = str(dct[page][tb]['text'])                   #convert the 'text' element to a string
                if (',' in txt):                                   #every name contains a comma
                    txt = remove_headings(txt)
                    lines = txt.split('\n')                        #split text into lines
                    words = lines[0].split()                       #split first line into words
                    for word in words:                             #loop through and strip out fund, acct, obj
                        if (word.isdigit()):
                            if (len(word)==4):                     # 4 digits means fund
                                fund = word
                            if (len(word)==8):                     # 8 digits means acct-code
                                acct = word
                            if (len(word)==5):                     # 5 digits means obj
                                obj = word
                            txt = txt[len(word)+1:]                # remove fund/acct/obj from txt
                    dnames[page][tb] = {}                          #initialize dictionary for this page
                    dnames[page][tb]['x0'] = dct[page][tb]['x0']
                    dnames[page][tb]['x1'] = dct[page][tb]['x1']
                    dnames[page][tb]['y0'] = dct[page][tb]['y0']
                    dnames[page][tb]['y1'] = dct[page][tb]['y1']
                    dnames[page][tb]['fund'] = fund
                    dnames[page][tb]['acct'] = acct
                    dnames[page][tb]['obj'] = obj
                    dnames[page][tb]['text'] = txt
    return(dnames)

### Consolidate text boxes that overlap on the vertical scale and contain names

In [11]:
def consolidate_name_boxes(names):
    newnames = {}
    
    for page in sorted(names.keys()):                                        #loop through pages of pdf
        newnames[page] = {}                                                  #initialize new names dictionary
        skip = make_array()                                                  #initialize list of boxes to skip
    
        for tb in sorted(names[page].keys()):                                #loop through text boxes on this page
            for tb2 in sorted(names[page].keys()):                           #compare this one to the others
                if ((tb2 > tb) & \
                    (names[page][tb]['y0'] <= names[page][tb2]['y1']) & \
                    (names[page][tb2]['y0'] <= names[page][tb]['y1'])):      
                    d = {}                                                   #initialize replacement entry
                    d['x0'] = names[page][tb]['x0']                          #keep x0    
                    d['x1'] = names[page][tb2]['x1']                         #replace x1 with tb2 value
                    d['y0'] = names[page][tb2]['y0']                         #replace y0 with tb2 value
                    d['y1'] = names[page][tb]['y1']                          #keep y1 value
                    d['text'] = names[page][tb]['text'] +\
                        names[page][tb2]['text']                             #contatenate text strings
                    d['fund'] = names[page][tb]['fund']                      #copy fund, acct, and obj
                    d['acct'] = names[page][tb]['acct']
                    d['obj'] = names[page][tb]['obj']
                    newnames[page][tb2] = d                                  #plug into dictionary
                    skip = np.append(skip,tb)                                #add old boxes to skip list
                    skip = np.append(skip,tb2)
            if (tb not in skip):                                             #if no match, check skip list 
                newnames[page][tb] = names[page][tb]                         #just copy if not in skip list
                    
    return(newnames)

In [12]:
def combdd(cn,pdf):
    
    dd = {}
    
    for page in sorted(cn.keys()):
        if page not in dd.keys():
            dd[page] = {}
        for tb in sorted(cn[page].keys()):                               #loop through consolidated name textboxes
            dd[page][tb] = cn[page][tb]
            y0  = dd[page][tb]['y0']                                      #extract vertical coordinates
            y1  = dd[page][tb]['y1']
            txt = dd[page][tb]['text']                           #extract text
            for tb2 in sorted(pdf[page].keys()):                            #loop through the other boxes in pdf
                if (tb != tb2):                                             #ignore if same box as names
                    tx0 = pdf[page][tb2]['x0']                              #get horizontal offset
                    ty0 = pdf[page][tb2]['y0']                              #check whether the vertical 
                    ty1 = pdf[page][tb2]['y1']                              #range of this box overlaps that
                    if ((y0 <= ty1) & (ty0 <= y1)):                         #of the name box
                        txt = remove_headings(pdf[page][tb2]['text'])
                        if ((312.0 < tx0) & (tx0 < 316.0)):                 #match to DATE/NUMBER
                            dd[page][tb]['numbers1'] = txt
                        if ((383.0 < tx0) & (tx0 < 395.0)):                 #match to NUMBER
                            if 'numbers2' not in dd[page][tb].keys():
                                dd[page][tb]['numbers2'] = txt
                            else:
                                dd[page][tb]['numbers2'] += txt
                        if ((437.0 < tx0) & (tx0 < 440.0)):                 #match to POSITION
                            dd[page][tb]['positions'] = txt
                        if ((509.0 < tx0) & (tx0 < 533.0)):                 #match to RATE 
                            dd[page][tb]['rates'] = remove_commas(txt)
                        if ((558.0 < tx0) & (tx0 < 630.0)):                 #match to ACCT-EARNINGS
                            dd[page][tb]['earnings'] = remove_commas(txt)

    return(dd)

In [13]:
def get_lines(nn):
    
    lld = {}
    
    for page in sorted(nn.keys()):
        if page not in lld.keys():
            lld[page] = {}
        for tb in sorted(nn[page].keys()):
            if tb not in lld[page].keys():
                lld[page][tb]              = {}
                lld[page][tb]['names']     = []
                lld[page][tb]['checks']    = []
                lld[page][tb]['dates']     = []
                lld[page][tb]['rates']     = []
                lld[page][tb]['earnings']  = []
                lld[page][tb]['positions'] = []
                lld[page][tb]['fund']      = ''
                lld[page][tb]['acct']      = ''
                lld[page][tb]['obj']       = ''
            txt = nn[page][tb]['text']
            words = txt.split('\n')
            for word in words:
                if (len(word) > 1):
                    lld[page][tb]['names'].append(word)
            if 'numbers1' in nn[page][tb].keys():
                txt = nn[page][tb]['numbers1']
                words = txt.split('\n')
                for word in words:
                    if word.isdigit():
                        lld[page][tb]['checks'].append(word)
                    elif '/' in word:
                        lld[page][tb]['dates'].append(word)
            if 'numbers2' in nn[page][tb].keys():
                txt = nn[page][tb]['numbers2']
                words = txt.split('\n')
                for word in words:
                    if word.isdigit():
                        lld[page][tb]['checks'].append(word)
            if 'rates' in nn[page][tb].keys():
                txt = nn[page][tb]['rates']
                words = txt.split('\n')
                for word in words:
                    if '.' in word:
                        lld[page][tb]['rates'].append(float(word))
            if 'positions' in nn[page][tb].keys():
                txt = nn[page][tb]['positions']
                words = txt.split('\n')
                for word in words:
                    if len(word)>1:
                        lld[page][tb]['positions'].append(word)
            if 'fund' in nn[page][tb].keys():
                lld[page][tb]['fund'] = nn[page][tb]['fund']
            if 'acct' in nn[page][tb].keys():
                lld[page][tb]['acct'] = nn[page][tb]['acct']
            if 'obj' in nn[page][tb].keys():
                lld[page][tb]['obj'] = nn[page][tb]['obj']
            if 'earnings' in nn[page][tb].keys():
                txt = nn[page][tb]['earnings']
                had_underscore = False
                words = txt.split('\n')
                for word in words:
                    if '.' in word:
                        if not had_underscore: 
                            lld[page][tb]['earnings'].append(float(word))
                            had_underscore = False
                    elif '_' in word:
                        had_underscore = True
            if (len(lld[page][tb]['checks']) < len(lld[page][tb]['dates'])):
                new_checks = []
                check_index = 0
                for i in np.arange(len(lld[page][tb]['earnings'])):
                    if (lld[page][tb]['earnings'][i] > 0.0):
                        new_checks.append(lld[page][tb]['checks'][check_index])
                        check_index += 1
                    else:
                        new_checks.append('gen'+str(page) + '-' + str(tb) + '-' + str(i))
                        print("inserting check number: ",page,tb,i)
                lld[page][tb]['checks'] = new_checks
    return(lld)

### Read the earnings reports and process them

In [14]:
def process_earnings(pdf):
    nnd = get_names(pdf)
    cnd = consolidate_name_boxes(nnd)
    newnames = combdd(cnd,pdf)
    lld = get_lines(newnames)
    return(lld)

ll={}

ll[2017] = process_earnings(p17)
ll[2018] = process_earnings(p18)

inserting check number:  262 6 39
inserting check number:  262 6 40
inserting check number:  43 6 30
inserting check number:  51 6 49
inserting check number:  94 7 19


### Check earnings against totals

In [15]:
totearn = {}

for year in ll.keys():
    if year not in totearn.keys():
        totearn[year] = 0.0
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            for amt in ll[year][page][tb]['earnings']:
                totearn[year] += amt
                
print(round(totearn[2017],2))       #FY2017 earnings report total is $22,608,024.34
print(round(totearn[2018],2))       #FY2018 earnings report total is $22,409,915.41

22608024.34
22409915.41


In [36]:
class check():                                                          #generic check class
    def __init__(self,check_number,name,check_date):         #constructor
        self.check_number   = check_number
        self.name           = name
        self.check_date     = check_date
        self.items          = []
        return

    def get_name(self):
        return(self.name)
        
    def get_date(self):
        return(self.check_date)
    
    def get_fiscal_year(self):
        y = self.check_date.year
        m = self.check_date.month
        if (m <= 6):
            return(y)
        else:
            return(y+1)
    
    def get_items(self):
        return(self.items)
    
    def add_item(self,fund,acct,obj,position,rate,earnings):
        self.items.append({'fund':fund,'acct':acct,'obj':obj,'position':position,'rate':rate,'earnings':earnings})
        return

In [37]:
class payperiod():                                      #class for dates at end of payperiods
    """Provides a dictionary of payroll periods"""
    
    def __init__(self):
    
        self.payperiods = {}                            #initialize payperiods dictionary

        delta = timedelta(days=14)                      #14 days per pay period
        first_payperiod = date(2009,7,3)                #first pay period of FY2010
        cutoff_date = date(2026,7,1)
        
        current_payperiod = first_payperiod
        current_year = first_payperiod.year
        current_month = first_payperiod.month

        while (current_payperiod < cutoff_date):         #generate pay periods through cutoff date
            self.payperiods[current_payperiod] = {}     #create empty dictionary for this payperiod
            fyear = current_payperiod.year              #compute fiscal year
            if (current_payperiod.month >= 7):          #for months7-12 it's current year plus 1
               fyear += 1
            self.payperiods[current_payperiod]['fyear'] = fyear
            schyear = current_payperiod.year
            if (current_payperiod.month < 8):
               schyear = str(current_payperiod.year-1) + '-' + str(current_payperiod.year)
            elif (current_payperiod.month > 8):
               schyear = str(current_payperiod.year) + '-' + str(current_payperiod.year+1)
            elif (current_payperiod.day < 14):
               schyear = str(current_payperiod.year-1) + '-' + str(current_payperiod.year)
            else:
               schyear = str(current_payperiod.year) + '-' + str(current_payperiod.year+1)
            self.payperiods[current_payperiod]['school_year'] = schyear
            self.payperiods[current_payperiod]['calendar_year'] = current_payperiod.year
            if ((current_payperiod.month == 6) & \
                ((current_payperiod+delta).month==7) & \
                ((current_payperiod+delta).day > 1)):
                self.payperiods[current_payperiod]['spans_fyear'] = 'True'
            else:
                self.payperiods[current_payperiod]['spans_fyear'] = 'False'
            current_payperiod += delta                                  #increment date by 14 days
        
        self.payperiods[date(2016,11,10)] = self.payperiods.pop(date(2016,11,11))
        return
    
    def get_payperiods(self):
        return(self.payperiods)
    
    def get_payperiod_end(self,fyear,ppno):                              #look up the date of the nth pay period
        """Lookup end date of payroll periods in a given fiscal year"""
        try:
            ppend = self.payperiods[fyear][ppno]
        except KeyError:
            ppend = np.NaN
            
        return(ppend)    
        
    def get_fiscal_year(self,xdate):                    #look up the fiscal year given a date
        """Lookup fiscal year given date"""
        fyr = xdate.year
        mon = xdate.month
        if (mon > 6):
            fyr += 1
        return(fyr)
    
    def get_school_year(self,xdate):                    #look up the fiscal year given a date
        """Lookup school year given date"""
        return(self.payperiods[xdate]['school_year'])
    
    def get_payperiod_no(self,fyear,xdate):             #look up the pay period number for a date
        """Lookup payroll period given a date"""
        period=1
        while self.payperiods[fyear][period] >= xdate:
            period += 1
        return(period)
    
    def get_next_payday(self,y,m,d):                                    #find the next payday after given date
        """Find the end of the current pay period given a date"""
        d = date(y,m,d)                                                 #convert y,m,d to date value
    
        for pdate in self.payperiods.keys():                            #look through paydates
            if (payday >= d):                                       #return the first one greater than 
                return(payday)                                      #the date supplied
            
        return(np.NaN)
    
    def get_previous_payday(self,cdate):                            #find the previous payday
        """Find the date of the previous payday"""
        
        tdate = np.NaN
        
        for pdate in sorted(self.payperiods.keys()):                #look through paydates
            if (pdate <= cdate):                                    #return the last one less than or equal
                tdate = pdate
            
        return(tdate)

In [38]:
pp = payperiod()          #get end dates of payroll periods
print('get_payperiod_end(fyear,ppno): ',pp.get_payperiod_end.__doc__)

get_payperiod_end(fyear,ppno):  Lookup end date of payroll periods in a given fiscal year


In [39]:
checks = {}

pp = payperiod()          #get end dates of payroll periods

paydays = pp.get_payperiods()

for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            check_numbers = ll[year][page][tb]['checks']
            names         = ll[year][page][tb]['names']
            check_dates   = ll[year][page][tb]['dates']
            fund          = ll[year][page][tb]['fund']
            acct          = ll[year][page][tb]['acct']
            obj           = ll[year][page][tb]['obj']
            positions     = ll[year][page][tb]['positions']
            rates         = ll[year][page][tb]['rates']
            earnings      = ll[year][page][tb]['earnings']
            
            for i in np.arange(len(check_numbers)):
                check_number = check_numbers[i]
                name         = names[i]
                date_str     = check_dates[i]
                words = date_str.split('/')
                check_date   = date(int(words[2]),int(words[0]),int(words[1]))
                if (check_date not in paydays.keys()):
                    new_date = pp.get_previous_payday(check_date)
                    print('adjusting date',name,check_date,new_date)
                    check_date = new_date
                if check_number not in checks.keys():
                    checks[check_number] = check(check_number,name,check_date)
                position = positions[i]
                rate     = rates[i]
                earned   = earnings[i]
                checks[check_number].add_item(fund,acct,obj,position,rate,earned)

adjusting date HARE, KIMBERLY C 2016-10-06 2016-09-30
adjusting date MALLILO, NATALIE A 2016-10-18 2016-10-14
adjusting date MALLILO, NATALIE A 2016-10-18 2016-10-14
adjusting date HARE, KIMBERLY C 2016-10-06 2016-09-30
adjusting date GARDINER, PATRICIA S 2018-05-02 2018-04-27


In [43]:
earns = {}

for check in checks.keys():
    check_date = checks[check].get_date()
    if check_date not in earns.keys():
        earns[check_date] = 0.0
    itms = checks[check].get_items()
    for item in itms:
        earns[check_date] += item['earnings']
    
for ckdate in sorted(earns.keys()):
    print(ckdate,round(earns[ckdate],2))
    

2016-07-08 712582.64
2016-07-22 731432.74
2016-08-05 727235.05
2016-08-19 830534.38
2016-09-02 873373.4
2016-09-16 851594.36
2016-09-30 894158.83
2016-10-14 909581.63
2016-10-28 872404.3
2016-11-10 873875.89
2016-11-25 867629.52
2016-12-09 927504.0
2016-12-23 876349.06
2017-01-06 1072459.77
2017-01-20 880519.3
2017-02-03 877020.3
2017-02-17 874339.35
2017-03-03 868450.13
2017-03-17 881158.86
2017-03-31 881394.06
2017-04-14 890815.44
2017-04-28 861520.55
2017-05-12 969064.7
2017-05-26 899585.29
2017-06-09 814842.99
2017-06-23 888597.8
2017-07-07 789836.2
2017-07-21 752795.33
2017-08-04 755545.28
2017-08-18 878423.53
2017-09-01 857147.41
2017-09-15 868564.35
2017-09-29 857886.53
2017-10-13 916538.88
2017-10-27 879381.42
2017-11-10 876286.0
2017-11-24 875194.05
2017-12-08 918455.79
2017-12-22 886606.47
2018-01-05 857416.1
2018-01-19 918743.59
2018-02-02 900223.44
2018-02-16 875292.52
2018-03-02 852273.64
2018-03-16 874380.51
2018-03-30 873770.06
2018-04-13 873510.17
2018-04-27 863265.62
2